# Using the LSST DM Stack in Python

This tutorial focuses on using the DM stack in Python.
The tasks we will go through in this notebook are more commonly executed as part of larger processing pipelines.
We execute them in a notebook here to give a peek into the pipelines and hopefully demonstrate how these pieces of the pipelines can be configured.

More notebook examples can be found here: https://github.com/RobertLuptonTheGood/notebooks/tree/master/Demos

## The Data Repository

Instead of operating directly on files and directories, we interact with on-disk data products via an abstraction layer called the *data butler*.
The butler operates on *data repositories*.
There is a small data repository generated from HyperSuprimeCam (HSC) public data located at `/project/shared/data/RSP_CHECK_REPO/` on all deployments of the Rubin Science Platform.
In this tutorial we'll focus on processing an individual image, and that's all this particular subset will support.

## Data Access with Butler

Data repositories are managed by an object called `Butler`.
To retrieve a dataset from the `Butler`, we start by constructing one pointing to the repository containing the data used in this tutorial:

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/project/shared/data/RSP_CHECK_REPO/butler.yaml')

We can then call `get` with the name and data ID of the dataset.
The name of the image that's saved directly after instrument signature removal (ISR) is `postISRCCD`.
We will put this and the code necessary to extract a sub region in a function that we can call later.

In [ ]:
def regen_exposure():
    exposure = butler.get("postISRCCD", exposure=1228, detector=87, instrument='HSC',
                          collections='HSC/defaults')

    bbox = exposure.getBBox()
    bbox.grow(-bbox.getDimensions()//3)  # box containing the central third (in each dimension)
    bbox.grow(-Extent2I(0, 400))  # make it a bit smaller in x
    # exposure[bbox] would also work here because exposure.getXY0() == (0, 0),
    # but it's dangerous in general because it ignores that origin.
    sub = Exposure(exposure, bbox=bbox, dtype=exposure.dtype, deep=False)
    return exposure, sub, bbox

## Image, Boxes, and (Crude) Image Display

A full 2k x 4k HSC CCD is a pretty big image to display when you don't have specialized display code.  The DM stack does have specialized display code, but it either requires DS9 (which requires some ssh tunnels to use with data living on a server) or a Firefly server installation. For this tutorial, we'll just throw together a naive matplotlib display function, and create a view to a subimage that we'll display instead of the full image.

This section features a few of our most important class objects:

- `lsst.afw.image.Exposure` is an image object that actually holds three image planes: the science image (`Exposure.image`), an image of variance in every pixel (`Exposure.variance`), an integer bit mask (`Exposure.mask`).  It also holds a lot of more complex objects that characterize the image, such as a point-spread function (`lsst.afw.detection.Psf`) and world-coordinate system (`lsst.afw.image.Wcs`).  Most of these objects aren't filled in yet, because all we've run so far is ISR.  It doesn't generally make sense to perform mathematical operations (i.e. addition) on `Exposure`s, because those operations aren't always well-defined on the more complex objects.  You can get a `MaskedImage` object with the same image, mask, and variance planes that does support mathematical operations but doesn't contain `Psf`s and `Wcs`s (etc) with `Exposure.maskedImage`.

- The `Exposure.image` and `Exposure.variance` properties return `lsst.afw.image.Image` objects.  These have a `.array` property that returns a `numpy.ndarray` view to the `Image`'s pixels.  Conceptually, you should think of an `Image` as just a `numpy.ndarray` with a possibly nonzero origin.

- The `Exposure.mask` property returns a `lsst.afw.image.Mask` object, which behaves like an `Image` with a dictionary-like object that relates string labels to bit numbers.

- All of these image-like objects have a `getBBox()` method, which returns a `lsst.afw.geom.Box2I`.  The minimum and maximum points of a `Box2I` are specified in integers that correspond to the *centers* of the lower-left and upper-right pixels in the box, but the box conceptually contains the entirety of those pixels.  To get a box with a floating-point representation of the same boundary for the `extent` argument to `imshow` below, we construct a `Box2D` from the `Box2I`.

- `Point2I` and `Extent2I` are used to represent absolute positions and offsets between positions as integers (respectively).  These have floating-point counterparts `Point2D` and `Extent2D`.

In [ ]:
from lsst.geom import Box2D, Box2I, Point2I, Extent2I
from lsst.afw.image import Exposure

In [ ]:
exposure, sub, bbox = regen_exposure()

In [ ]:
import numpy as np
import matplotlib
%matplotlib inline
matplotlib.rcParams["figure.figsize"] = (8, 6)
matplotlib.rcParams["font.size"] = 12

In [ ]:
def display(image, mask=None, colors=None, alpha=0.40, **kwds):
    box = Box2D(image.getBBox())
    extent = (box.getMinX(), box.getMaxX(), box.getMinY(), box.getMaxY())
    kwds.setdefault("extent", extent)
    kwds.setdefault("origin", "lower")
    kwds.setdefault("interpolation", "nearest")
    matplotlib.pyplot.imshow(image.array, **kwds)
    kwds.pop("vmin", None)
    kwds.pop("vmax", None)
    kwds.pop("norm", None)
    kwds.pop("cmap", None)
    if mask is not None:
        for plane, color in colors.items():
            array = np.zeros(mask.array.shape + (4,), dtype=float)
            rgba = np.array(matplotlib.colors.hex2color(matplotlib.colors.cnames[color]) + (alpha, ),
                            dtype=float)
            np.multiply.outer((mask.array & mask.getPlaneBitMask(plane)).astype(bool), rgba, out=array)
            matplotlib.pyplot.imshow(array, **kwds)

And now here's the (cutout) of the detrended image.  I've cheated in setting the scale by looking at the background level in advance.

In [ ]:
display(sub.image, vmin=3300, vmax=3500, cmap=matplotlib.cm.gray)

## Background Subtraction and Task Configuration

The next step we usually take is to estimate and subtract the background, using `lsst.meas.algorithms.SubtractBackgroundTask`.  This is a regular `Task`, not a `CmdLineTask`, and hence we'll just pass it our `Exposure` object (it operates in-place) instead of a `Butler`.

In [ ]:
from lsst.meas.algorithms import SubtractBackgroundTask

In [ ]:
bkgConfig = SubtractBackgroundTask.ConfigClass()

In [ ]:
# Execute this cell to get fun & terrible results!
bkgConfig.useApprox = False
bkgConfig.binSize = 20

The pattern for configuration here is the same as it was for `SubaruIsrTask`, but here we're setting values directly instead of loading a configuration file from the `obs_subaru` camera-specialization package.  The `config` object here is an instance of a class that inherits from `lsst.pex.config.Config` that contains a set of `lsst.pex.config.Field` objects that define the options that can be modified.  Each `Field` behaves more or less like a Python `property`, and you can get information on all of the fields in a config object by either using `help`:

In [ ]:
help(bkgConfig)

In [ ]:
SubtractBackgroundTask.ConfigClass.algorithm?

In [ ]:
bkgTask = SubtractBackgroundTask(config=bkgConfig)

In [ ]:
bkgResult = bkgTask.run(exposure)

In [ ]:
display(sub.image, vmin=-0.5, vmax=100, cmap=matplotlib.cm.gray)

If you've run through all of these steps after executing the cell that warns about terrible results, you should notice that the galaxy in the upper right has been oversubtracted.

**EXERCISE**: Before continuing on, re-load the exposure from disk, reset the configuration and `Task` instances, and re-run without executing the cell that applies bad values to the config, all by just re-executing the right cells above.  You should end up an image in which the upper-right galaxy looks essentially the same as it does in the image before we subtracted the background.

**ANSWER:**

In [ ]:
exposure, sub, bbox = regen_exposure()

We could construct a new instance of `SubtractBackgroundTask.ConfigClass` and pass it unmodified to the constructor (and that's what you'd get by strictly copying existing cells), but it also works to just construct the task with no arguments.

In [ ]:
bkgTask = SubtractBackgroundTask()

In [ ]:
bkgResult = bkgTask.run(exposure)

## Installing an Initial-Guess PSF

Most later processing steps require a PSF model, which is represented by a `Psf` object that's attached to the `Exposure`.  For now, we'll just make a Gaussian PSF with some guess at the seeing.

In [ ]:
from lsst.meas.algorithms import SingleGaussianPsf

In [ ]:
FWHM_TO_SIGMA = 1.0/(2*np.sqrt(2*np.log(2)))
PIXEL_SCALE = 0.168  # arcsec/pixel
SEEING = 0.7         # FWHM in arcsec
sigma = FWHM_TO_SIGMA*SEEING/PIXEL_SCALE
width = int(sigma*3)*2 + 1
psf = SingleGaussianPsf(width, width, sigma=sigma)
exposure.setPsf(psf)

A `Psf` object can basically just do one thing: it can return an image of itself at a point.  `SingleGaussianPsf` represents a constant PSF, so it always returns the same image, regardless of the point you give it.

But there are two ways to evaluate a `Psf` at a point.  If you want an image centered on the middle pixel, and that middle pixel to be the origin - what you'd usually want if you're going to convolve the PSF with another model - use `computeKernelImage(point)`:

In [ ]:
from lsst.geom import Point2D
display(psf.computeKernelImage(Point2D(60.5, 7.2)))

If you want to compare the PSF to a star at the exact same position, use `computeImage(point)`.  That will shift the image returned by `computeKernelImage(point)` to the right sub-pixel offset, and update the origin of the image to take care of the rest, so you end up with a postage stamp in the same coordinate system as the original image where the star is.

In [ ]:
display(psf.computeImage(Point2D(60.5, 7.2)))

## Removing Cosmic Rays

Cosmic rays are detected and interpolated by `RepairTask`, which also sets mask planes to indicate where the cosmic rays were ("CR") and which pixels were interpolated ("INTERP"; this may happen due to saturation or bad pixels as well).  Because we're just using the default configuration, we can skip creating a config object and just construct the `Task` with no arguments.

In [ ]:
from lsst.pipe.tasks.repair import RepairTask

In [ ]:
repairTask = RepairTask()

In [ ]:
repairTask.run(exposure)

In [ ]:
display(sub.image, mask=sub.mask, colors={"CR": "red"},
        vmin=-0.5, vmax=100, alpha=0.8, cmap=matplotlib.cm.gray)

## Detecting Sources

Unlike the other `Task`s we've dealt with so far, `SourceDetectionTask` creates a `SourceCatalog` in addition to updating the image (all it does to the image is add a "DETECTED" mask plane).  All `Task`s that work with catalogs need to be initialized with a `lsst.afw.table.Schema` object, to which the `Task` will add the fields necessary to store its outputs.  A `SourceCatalog`'s `Schema` cannot be modified after the `SourceCatalog` has been constructed, which means it's necessary to construct all `Schema`-using `Task`s before actually running any of them.

Each record in the catalog returned by `SourceDetectionTask` has a `Footprint` object attached to it.  A `Footprint` represents the approximate region covered by a source in a run-length encoding data structure.  It also contains a list of peaks found within that region.  The "DETECTED" mask plane is set to exactly the pixels covered by any `Footprint` in the returned catalog.

In [ ]:
from lsst.meas.algorithms import SourceDetectionTask
from lsst.afw.table import SourceTable, SourceCatalog

In [ ]:
schema = SourceTable.makeMinimalSchema()

In [ ]:
detectTask = SourceDetectionTask(schema=schema)

In [ ]:
# A SourceTable is really just a factory object for records; don't confuse it with SourceCatalog, which is
# usually what you want.  But a SourceTable *is* what SourceDetectionTask wants here.
table = SourceTable.make(schema)

In [ ]:
detectResult = detectTask.run(table, exposure)

In [ ]:
display(sub.image, mask=sub.mask, colors={"DETECTED": "blue"}, vmin=-0.5, vmax=100, cmap=matplotlib.cm.gray)

## Deblending

Deblending attempts to separate detections with multiple peaks into separate objects.  We keep all of the original sources in the `SourceCatalog` (called `parent`s) when we deblend, but for each `parent` source that contains more than one peak, we create a new record (called a `child`) for each of those peaks.  The `Footprint`s attached to the `child` objects are instances of a subclass called `HeavyFootprint`, which include new deblended pixel values as well as the region description.  These can be used by calling `insert` to replace an `Image`'s pixels with the `HeavyFootprint`'s pixels.

**EXERCISE**: This section will not run if the cells are executed naively in order.  At some point you'll have to go re-execute one or more cells in the previous section to get the right behavior.  Which one(s)?  Why?  Copy those cells here (in the right places) when you figure it out.

**ANSWER:**

In [ ]:
from lsst.meas.deblender import SourceDeblendTask

It isn't necessary to re-load the `Exposure` from disk (and re-do background subtraction) to get this exercise to run, but not doing so will result in re-subtracting the residual background repeatedly every time `SourceDetectionTask` is run, which means the results will change slightly each time.

In [ ]:
exposure, sub, bbox = regen_exposure()
exposure.setPsf(psf)

It isn't necessary to construct all of these tasks before running any of them (just the ones that take and modify a schema), but it's easier to get things right if you follow that principle.

In [ ]:
schema = SourceTable.makeMinimalSchema()
bkgTask = SubtractBackgroundTask()
repairTask = RepairTask()
detectTask = SourceDetectionTask(schema=schema)

In [ ]:
deblendTask = SourceDeblendTask(schema=schema)

Now we run the tasks from the previous sections:

In [ ]:
bkgResult = bkgTask.run(exposure)
repairTask.run(exposure)
table = SourceTable.make(schema)
detectResult = detectTask.run(table, exposure)

And, finally, we can run the task introduced in this section.

In [ ]:
catalog = detectResult.sources

In [ ]:
deblendTask.run(exposure, catalog)

To inspect some deblender outputs, we'll start by finding some parent objects that were deblended into multiple children, by looking at the `deblend_nChild` field (which was added to the `Schema` when we constructed the `SourceDeblendTask`, and populated when we called `run`).

In [ ]:
# Find some blended sources inside the subimage:
blendParents = []
for record in catalog:
    if record.get("deblend_nChild") > 0 and bbox.contains(record.getFootprint().getBBox()):
        blendParents.append(record)
# Sort by peak brightness so we can look at something with decent S/N
blendParents.sort(key=lambda r: -r.getFootprint().getPeaks()[0].getPeakValue())

In [ ]:
from lsst.afw.image import Image

The image of the parent object is just the original image, but we'll cut out just the region inside its `Footprint`:

In [ ]:
blendParentImage = Image(exposure.image, bbox=blendParents[0].getFootprint().getBBox(),
                         deep=True, dtype=np.float32)

Now we'll insert the deblended child pixels into blank images of the same size:

In [ ]:
blendChildImages = []
for blendChild in catalog.getChildren(blendParents[0].getId()):
    image = Image(blendParentImage.getBBox(), dtype=np.float32)
    blendChild.getFootprint().insert(image)
    blendChildImages.append(image)

In [ ]:
nSubPlots = len(blendChildImages) + 1
nCols = 3
nRows = nSubPlots//nCols + 1
matplotlib.pyplot.subplot(nRows, nCols, 1)
display(blendParentImage, vmin=-0.5, vmax=100, cmap=matplotlib.cm.gray)
for n, image in enumerate(blendChildImages):
    matplotlib.pyplot.subplot(nRows, nCols, n + 2)
    display(image, vmin=-0.5, vmax=100, cmap=matplotlib.cm.gray)

## Measurement

`SingleFrameMeasurementTask` is typically responsible for adding most fields to a `SourceCatalog`.  It runs a series of plugins that make different measurements (you can configure them with the `.plugins` dictionary-like field on its config object, and control which are run with `.names`).  If the deblender has been run first, it will measure child objects using their deblended pixels.

**EXERCISE**: Like the Deblending section, you'll have to re-execute some previous cells somewhere in this section to get the right behavior.  Copy those cells into the right places here once you've gotten it working.

**ANSWER:** As in the Deblending section exercise, we reload the exposure, create all of the tasks, and then run all of the tasks.

In [ ]:
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
exposure, sub, bbox = regen_exposure()
exposure.setPsf(psf)

In [ ]:
schema = SourceTable.makeMinimalSchema()
bkgTask = SubtractBackgroundTask()
repairTask = RepairTask()
detectTask = SourceDetectionTask(schema=schema)
deblendTask = SourceDeblendTask(schema=schema)

In [ ]:
measureConfig = SingleFrameMeasurementTask.ConfigClass()

In [ ]:
# What measurements are configured to run
print(measureConfig.plugins.names)

In [ ]:
# Import an extension module that adds a new measurement
import lsst.meas.extensions.photometryKron

In [ ]:
# What measurements *could* be configured to run
print(list(measureConfig.plugins.keys()))

In [ ]:
# Configure the new measurement to run
measureConfig.plugins.names.add("ext_photometryKron_KronFlux")

In [ ]:
measureTask = SingleFrameMeasurementTask(schema=schema, config=measureConfig)

In [ ]:
bkgResult = bkgTask.run(exposure)
repairTask.run(exposure)
table = SourceTable.make(schema)
detectResult = detectTask.run(table, exposure)
catalog = detectResult.sources
deblendTask.run(exposure, catalog)

In [ ]:
measureTask.run(catalog, exposure)

We'll show some of the results of measurement by overlaying the measured ellipses on the image.

The shapes and centroids we use here (by calling `record.getX()`, `record.getY()`, `record.getShape()`) are aliases  (called "slots") to fields with longer names that are our recommended measurements for these quantities.  You can see the set of aliases by printing the schema (see next section).

In [ ]:
from lsst.afw.geom.ellipses import Axes

In [ ]:
display(sub.image, mask=sub.mask, colors={"DETECTED": "blue"}, vmin=-0.5, vmax=100, cmap=matplotlib.cm.gray)

for record in catalog:
    if record.get("deblend_nChild") != 0:
        continue
    axes = Axes(record.getShape())   # convert to A, B, THETA parameterization
    axes.scale(2.0)  # matplotlib uses diameters, not radii
    # Don't generate ellipses for records with nans
    nans = np.isnan([record.getX(), record.getY(), axes.getA(), axes.getB(), axes.getTheta()])
    if sum(nans) > 0:
        continue
    patch = matplotlib.patches.Ellipse((record.getX(), record.getY()),
                                       axes.getA(), axes.getB(), axes.getTheta() * 180.0 / np.pi,
                                      fill=False, edgecolor="green")
    matplotlib.pyplot.gca().add_patch(patch)
matplotlib.pyplot.show()

## Working With Catalogs

Print the schema:

In [ ]:
print(catalog.getSchema())

Get arrays of columns (requires the catalog to be continguous in memory, which we can guarantee with a deep copy):

In [ ]:
catalog = catalog.copy(deep=True)

In [ ]:
psfFlux = catalog["base_PsfFlux_instFlux"]

Note that boolean values are stored in `Flag` columns, which are packed into bits.  Unlike other column types, when you get an array of a `Flag` column, you get a copy, not a view.

Use `Key` objects instead of strings to do fast repeated access to fields when iterating over records:

In [ ]:
key = catalog.getSchema().find("deblend_nChild").key
deblended = [record for record in catalog if record.get(key) == 0]

You can also get `dict` version of a subset of a `Schema`, a `Catalog`, or a `Record` by calling either `extract` methods with a glob:

In [ ]:
catalog[0].extract("base_PsfFlux_*")  # or regex='...'

For `Record`s, the dict values are just the values of the fields, and for `Catalogs`, they're `numpy.ndarray` columns.  For `Schema`s they're `SchemaItem`s, which behave liked a named tuple containing a `Key` and a `Field`, which contains more descriptive information.

Get an Astropy view of the catalog (from which you can make a Pandas view):

In [ ]:
table = catalog.asAstropy()

You can find some reference documentation for the catalog library [here](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/afw_table.html).

# Exercises

1) Make a scatter plot of Kron Flux vs. Psf Flux.

It's not a pretty plot, with so few points and no calibration, but it's pretty simple:

In [ ]:
matplotlib.pyplot.scatter(catalog["slot_PsfFlux_instFlux"], catalog["ext_photometryKron_KronFlux_instFlux"], alpha=0.5)

2) Write a single function that performs all of the above steps on a post-ISR `Exposure` object, modifying the `Exposure` in-place and returning a new `SourceCatalog` with a complete set of measurements.

In [ ]:
def processExposure1(exposure):
    schema = SourceTable.makeMinimalSchema()
    bkgTask = SubtractBackgroundTask()
    repairTask = RepairTask()
    detectTask = SourceDetectionTask(schema=schema)
    deblendTask = SourceDeblendTask(schema=schema)
    measureConfig = SingleFrameMeasurementTask.ConfigClass()
    measureConfig.plugins.names.add("ext_photometryKron_KronFlux")
    measureTask = SingleFrameMeasurementTask(schema=schema, config=measureConfig)
    bkgResult = bkgTask.run(exposure)
    repairTask.run(exposure)
    table = SourceTable.make(schema)
    detectResult = detectTask.run(table, exposure)
    catalog = detectResult.sources
    deblendTask.run(exposure, catalog)
    measureTask.run(catalog, exposure)
    return catalog

In [ ]:
exposure, sub, bbox = regen_exposure()
exposure.setPsf(psf)

In [ ]:
catalog = processExposure1(exposure)

3) Add PSF modeling to the end of that function, delegating most of the work to `lsst.pipe.tasks.MeasurePsfTask`.  You may want to use a higher threshold (e.g. 50-sigma) for detection, since PSF modeling should only use bright stars.  Hint: `MeasurePsfTask` requires the catalog you pass it to be contiguous; if it isn't you'll (unfortunately) get a very confusing error message.

In [ ]:
from lsst.pipe.tasks.measurePsf import MeasurePsfTask

def processExposure2(exposure):
    schema = SourceTable.makeMinimalSchema()
    bkgTask = SubtractBackgroundTask()
    repairTask = RepairTask()
    detectConfig = SourceDetectionTask.ConfigClass()
    detectConfig.thresholdValue = 50.0
    detectTask = SourceDetectionTask(config=detectConfig, schema=schema)
    deblendTask = SourceDeblendTask(schema=schema)
    measureConfig = SingleFrameMeasurementTask.ConfigClass()
    measureConfig.plugins.names.add("ext_photometryKron_KronFlux")
    measureTask = SingleFrameMeasurementTask(schema=schema, config=measureConfig)
    psfTask = MeasurePsfTask(schema=schema)
    bkgResult = bkgTask.run(exposure)
    repairTask.run(exposure)
    table = SourceTable.make(schema)
    detectResult = detectTask.run(table, exposure)
    catalog = detectResult.sources
    deblendTask.run(exposure, catalog)
    catalog = catalog.copy(deep=True)
    measureTask.run(catalog, exposure)
    psfTask.run(exposure, catalog)
    return catalog

In [ ]:
exposure, sub, bbox = regen_exposure()
exposure.setPsf(psf)

In [ ]:
catalog = processExposure2(exposure)

4) Make images of the PSF stars, the PSF model at the position of those stars, and the difference between them.

In [ ]:
from lsst.afw.image import PARENT

stamps = []
for record in catalog[catalog["calib_psf_used"]]:
    # Get the PSF, convert to a single-precision image (yes, ugly syntax)
    psfImage = exposure.getPsf().computeImage(record.getCentroid()).convertF()
    # Make a subimage (yes, even uglier syntax).  Last boolean is whether to make a deep copy.
    starImage = exposure.image.Factory(exposure.image, psfImage.getBBox(), PARENT, True)
    # Divide the star image by the PSF flux (note that this wouldn't work as well if we had applied aperture corrections)
    starImage /= record.get("slot_PsfFlux_instFlux")
    # Make a deep copy, and subtract to get the residuals
    residualImage = starImage.clone()
    residualImage -= psfImage
    stamps.append((psfImage, starImage, residualImage))

In [ ]:
limit = 4
nCols = 3
nRows = min(len(stamps), limit)
index = 1
for psfImage, starImage, residualImage in stamps[:nRows]:
    vmin = min(psfImage.array.min(), starImage.array.min(), residualImage.array.min())
    vmax = max(psfImage.array.max(), starImage.array.max(), residualImage.array.max())
    matplotlib.pyplot.subplot(nRows, nCols, index)
    display(starImage, vmin=vmin, vmax=vmax)
    index += 1
    matplotlib.pyplot.subplot(nRows, nCols, index)
    display(psfImage, vmin=vmin, vmax=vmax)
    index += 1
    matplotlib.pyplot.subplot(nRows, nCols, index)
    display(residualImage, vmin=vmin, vmax=vmax)
    index += 1

Clearly the PSF model we get without tuning any of the configuration for `MeasurePsfTask` isn't terribly good.

5) Add another detect-deblend-measure sequence after PSF modeling at a deeper threshold.

In [ ]:
from lsst.pipe.tasks.measurePsf import MeasurePsfTask

def processExposure3(exposure):
    # Construct tasks and schema for the first pass to do PSF estimation
    schema1 = SourceTable.makeMinimalSchema()
    bkgTask = SubtractBackgroundTask()
    repairTask = RepairTask()
    detectConfig1 = SourceDetectionTask.ConfigClass()
    detectConfig1.thresholdValue = 50.0
    detectTask1 = SourceDetectionTask(config=detectConfig1, schema=schema1)
    deblendTask1 = SourceDeblendTask(schema=schema1)
    measureTask1 = SingleFrameMeasurementTask(schema=schema1)
    psfTask = MeasurePsfTask(schema=schema1)
    # Construct tasks and schema for the second pass
    schema2 = SourceTable.makeMinimalSchema()
    detectTask2 = SourceDetectionTask(schema=schema2)
    deblendTask2 = SourceDeblendTask(schema=schema2)
    measureConfig2 = SingleFrameMeasurementTask.ConfigClass()
    measureConfig2.plugins.names.add("ext_photometryKron_KronFlux")
    measureTask2 = SingleFrameMeasurementTask(schema=schema2, config=measureConfig2)
    # Run the first pass, and do PSF estimation.
    bkgResult = bkgTask.run(exposure)
    repairTask.run(exposure)
    table1 = SourceTable.make(schema1)
    detectResult1 = detectTask1.run(table1, exposure)
    catalog1 = detectResult1.sources
    deblendTask1.run(exposure, catalog1)
    catalog1 = catalog1.copy(deep=True)
    measureTask1.run(catalog1, exposure)
    # Run the second pass
    table2 = SourceTable.make(schema2)
    detectResult2 = detectTask2.run(table2, exposure)
    catalog2 = detectResult2.sources
    deblendTask2.run(exposure, catalog2)
    measureTask2.run(catalog2, exposure)
    return catalog1, catalog2

In [ ]:
exposure, sub, bbox = regen_exposure()
exposure.setPsf(psf)

In [ ]:
catalog1, catalog2 = processExposure3(exposure)

6) Rewrite the function as a class that constructs all of the `Task`s that it use in `__init__` and processes a single `Exposure` with you call its `run` method.  Make sure it will behave properly if `run` is called multiple times wiht different `Exposure` objects.

In [ ]:
class ProcessExposure(object):
    
    def __init__(self):
        # Construct tasks and schema for the first pass to do PSF estimation
        self.schema1 = SourceTable.makeMinimalSchema()
        self.bkgTask = SubtractBackgroundTask()
        self.repairTask = RepairTask()
        detectConfig1 = SourceDetectionTask.ConfigClass()
        detectConfig1.thresholdValue = 50.0
        self.detectTask1 = SourceDetectionTask(config=detectConfig1, schema=self.schema1)
        self.deblendTask1 = SourceDeblendTask(schema=self.schema1)
        self.measureTask1 = SingleFrameMeasurementTask(schema=self.schema1)
        self.psfTask = MeasurePsfTask(schema=self.schema1)
        # Construct tasks and schema for the second pass
        self.schema2 = SourceTable.makeMinimalSchema()
        self.detectTask2 = SourceDetectionTask(schema=self.schema2)
        self.deblendTask2 = SourceDeblendTask(schema=self.schema2)
        measureConfig2 = SingleFrameMeasurementTask.ConfigClass()
        measureConfig2.plugins.names.add("ext_photometryKron_KronFlux")
        self.measureTask2 = SingleFrameMeasurementTask(schema=self.schema2, config=measureConfig2)
        
    def run(self, exposure):
        # Run the first pass, and do PSF estimation.
        bkgResult = self.bkgTask.run(exposure)
        self.repairTask.run(exposure)
        table1 = SourceTable.make(self.schema1)
        detectResult1 = self.detectTask1.run(table1, exposure)
        catalog1 = detectResult1.sources
        self.deblendTask1.run(exposure, catalog1)
        catalog1 = catalog1.copy(deep=True)
        self.measureTask1.run(catalog1, exposure)
        # Run the second pass
        table2 = SourceTable.make(self.schema2)
        detectResult2 = self.detectTask2.run(table2, exposure)
        catalog2 = detectResult2.sources
        self.deblendTask2.run(exposure, catalog2)
        self.measureTask2.run(catalog2, exposure)
        return catalog1, catalog2

In [ ]:
exposure, sub, bbox = regen_exposure()
exposure.setPsf(psf)

In [ ]:
task = ProcessExposure()
catalog1, catalog2 = task.run(exposure)


- 